In [1]:
import sys
import os
import json
sys.path.append('..')
# sys.path.append('datasets')
# sys.path.append('models')
import pandas as pd

from models.propbank_encoder import PropbankEncoder
from models.conll_evaluator import ConllEvaluator

# this is the default configuration
default = {"batch_size": 250,
          "chunks": False,
          "ctxp": 1,
          "embeddings_model": "glo50",
          "embeddings_trainable": False,           
          "epochs": 1000,
          "hidden_layers": [32],
          "input_labels": ["ID", "FORM", "MARKER", "GPOS", "FORM_CTX_P-1", "FORM_CTX_P+0", "FORM_CTX_P+1"],
          "lr": 0.005,
          "ru": "BasicLSTM",
          "target_labels": ["T"],
          "kfold": False,
          "version": "1.0"}

# initialize PropbankEncoder
pe = PropbankEncoder.recover('../datasets/binaries/1.0/deep_glo50.pickle')

# initialize ConllEvaluator
ev = ConllEvaluator(pe, target_dir=os.getcwd())



In [2]:
def deep_leaderboard(path, file_list):
    filter_tuple = ('outputs', '1.0', '..')
    params_list = [hp for hp in path.split('/') if hp not in filter_tuple]
  
    params_dict = default.copy()
    if 'params.json' in file_list:
        # First attempt to find the params files
        file_path = path + '/params.json'
        with open(file_path, mode='r') as f:
            params_dict.update(json.load(f))
    else:
        def get_hidden(hidden_string):
            try:
                hidden_list = hidden_string.split('_')[-1].split('x')
                hidden_list = [int(x) for x in hidden_list]
            except ValueError:
                import code; code.interact(local=dict(globals(), **locals()))
            return hidden_list
        
        # Get data from parameters string
        params_dict['embeddings_model'] = params_list[0]
        params_dict['hidden_layers'] = get_hidden(params_list[1])
        params_dict['ctxp'] = int(params_list[2].split('_')[-1])
        params_dict['target_labels'] = params_list[3].split('_')
        params_dict['kfold'] = params_list[4] == 'kfold'
        params_dict['lr'] = float(params_list[5].split('_')[-1])
    
    params_dict['timestamp'] = params_list[6]
    # Get train F1
    if 'train.conll' in file_list:
        ev.evaluate_fromconllfile(path + '/train.conll')
        train_f1 = ev.f1
    else:
        train_f1 = 0.0    
    params_dict['train-f1'] = train_f1

    # Get valid F1
    if 'best-valid.conll' in file_list:
        ev.evaluate_fromconllfile(path + '/best-valid.conll')
        valid_f1 = ev.f1
    elif 'valid.conll' in file_list:
        ev.evaluate_fromconllfile(path + '/valid.conll')
        valid_f1 = ev.f1
    else:
        valid_f1 = 0.0
    
    params_dict['valid-f1'] = valid_f1
    print(train_f1, valid_f1)

    return params_dict
        
            

In [4]:
# Look into outputs
dir_path = '../outputs/1.0/'
zip_list = [(path_, file_list) for path_, subpath_, file_list in os.walk(dir_path) if not subpath_]
exp_list = [deep_leaderboard(*args) for args in zip_list]

df = pd.DataFrame(exp_list)




57.25 39.96
62.42 46.07
61.5 46.86
49.47 41.08
71.85 47.72
2.74 3.87
77.54 51.12
68.26 49.11
83.38 49.24
88.38 49.72
66.11 48.55
81.81 50.39
76.71 50.05
0.0 0.0
0.0 0.0
0.0 0.0
65.7 49.52
79.95 46.64
59.76 45.79
95.65 46.4
64.0 47.56
66.38 46.85
98.26 47.79
98.68 64.98
98.3 46.8
0.0 28.34
0.0 26.02
0.0 58.59
0.0 53.83
0.0 33.48
0.0 48.77
87.66 47.95
92.33 50.77
79.92 50.47
61.91 41.12
58.36 46.78
90.83 47.96
35.62 32.3
99.73 50.5
70.13 47.18
71.1 50.78
81.28 48.21
81.9 50.05
31.24 30.5
98.42 48.6
99.76 44.22
88.09 47.59
99.76 47.5
80.18 51.61
99.72 47.4
74.37 49.08
83.81 53.37
82.25 51.29
74.82 50.69
84.43 52.44
84.24 49.13
99.97 52.57
32.99 26.26
0.0 0.0
98.68 49.57
75.95 47.57
0.0 0.0
0.0 0.0
98.84 45.23
61.23 50.42
62.48 40.96
75.58 50.05
74.83 54.27
0.0 0.0
74.78 49.13
61.57 51.09
74.64 54.31
0.0 0.0
99.62 51.85
84.0 56.04
0.0 0.0
91.79 51.65
66.11 49.29
98.36 49.11
43.44 39.35
58.0 46.73
0.0 0.0
98.59 48.62
99.03 48.89
84.68 48.78
99.58 50.27
99.57 50.4
98.44 48.88
90.79 45.15
94.

In [5]:
df.reset_index(drop=True,inplace=True)
df.sort_values('valid-f1', ascending=False, inplace=True)
df.head()

,batch_size,chunks,ctxp,embeddings_model,embeddings_trainable,epochs,hidden_layers,input_labels,kfold,lr,ru,target_labels,timestamp,train-f1,valid-f1,version
23,250,False,1,glo50,False,1000,[32],"[ID, FORM, MARKER, GPOS, FORM_CTX_P-1, FORM_CT...",False,0.005,BasicLSTM,[T],2018-09-22 130752,98.68,64.98,1.0
27,250,False,1,glo50,False,1000,[32],"[ID, FORM, MARKER, GPOS, FORM_CTX_P-1, FORM_CT...",25,0.005,BasicLSTM,[T],2018-09-04 160737,0.00,58.59,1.0
74,100,False,1,wan50,False,150,"[16, 16]","[ID, FORM, MARKER, GPOS, FORM_CTX_P-1, FORM_CT...",False,0.005,BasicLSTM,"[R, IOB]",2018-09-26 174440,84.00,56.04,1.0
71,250,False,1,wan50,False,250,"[16, 16]","[ID, FORM, MARKER, GPOS, FORM_CTX_P-1, FORM_CT...",False,0.005,LSTM,"[R, IOB]",2018-09-29 230549,74.64,54.31,1.0
67,250,False,1,wan50,False,250,"[16, 16]","[ID, FORM, MARKER, GPOS, FORM_CTX_P-1, FORM_CT...",False,0.005,GRU,"[R, IOB]",2018-09-28 175327,74.83,54.27,1.0


In [6]:
df.to_csv('leaderboard.csv', sep=';', encoding='utf-8')